In [1]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

class smartCoarseGrainDataset(Dataset):
    def __init__(self, dataDir, transform=None):
        self.data = ImageFolder(dataDir, transform=transform)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        return x
    @property
    def classes(self):
        return self.data.classes
    


In [ ]:
#transforms = transforms.Compose([transforms.ToTensor()])
data = smartCoarseGrainDataset(dataDir = r"C:\Users\pgott\Desktop\Coding\Dataset\\", transform=transforms)
dataLoader = DataLoader(data, batch_size=1, shuffle=True, num_workers=0)


In [14]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, upscale_factor):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(32, 1 * (upscale_factor ** 2), (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

    def forward(self, x):
        x = F.tanh(self.conv1(x))
        x = F.tanh(self.conv2(x))
        x = F.sigmoid(self.pixel_shuffle(self.conv3(x)))
        return x


In [16]:
for images, labels in dataLoader:
   Net = Net(upscale_factor=3)
   output = Net.forward(torch.mean(images, dim=1, keepdim=False))
   break

TypeError: Net.forward() got an unexpected keyword argument 'upscale_factor'